# Demo of MURA-Subset Classification Using RADTorch

## Install RADTorch

In [ ]:
pip install git+https://download.radtorch.com/ -q

## Import RADTorch

In [ ]:
from radtorch import pipeline, core
from radtorch.settings import *

## Data Prep

In [ ]:
data_root = '/kaggle/input/mura-v11/MURA-v1.1/train_image_paths.csv'

In [ ]:
x = pd.read_csv(data_root, header=None, names=['PATH'])
x 

In [ ]:
shoulder_files = []
shoulder_labels = []
for i, r in x.iterrows():
    if 'SHOULDER' in r['PATH']:
        shoulder_files.append('/kaggle/input/mura-v11/'+r['PATH'])
        if 'positive' in r['PATH']:
            shoulder_labels.append('POSITIVE')
        else:
            shoulder_labels.append('NEGATIVE')
               


In [ ]:
shoulder_data = pd.DataFrame(zip(shoulder_files,shoulder_labels), columns=['PATH', 'LABEL'])
shoulder_data

## Classifier #1 : 
A RESNET50 feature extractor followed by xgboost classifier 5-fold cross validation

In [ ]:
clf = pipeline.Image_Classification(data_directory = '/kaggle/input/mura-v11/MURA-v1.1/train/', 
                                    table=shoulder_data, 
                                    is_dicom=False,
                                    is_path=True,
                                    image_path_column='PATH', image_label_column='LABEL',
                                    sampling=0.5,
                                    model_arch='resnet50',
                                    type='xgboost',
                                    parameters={'tree_method':'gpu_hist'})

In [ ]:
clf.data_processor.sample()

In [ ]:
clf.data_processor.dataset_info()

In [ ]:
clf.run()

In [ ]:
clf.classifier.roc()

In [ ]:
clf.classifier.confusion_matrix()

## Classifier #2 : 
A RESNET50 neural network trained for 5 epochs using transfer learning

In [ ]:
clf2 = pipeline.Image_Classification(data_directory = '/kaggle/input/mura-v11/MURA-v1.1/train/', 
                                    table=shoulder_data, 
                                    is_dicom=False,
                                    is_path=True,
                                    image_path_column='PATH', image_label_column='LABEL',
                                    sampling=0.5,
                                    model_arch='resnet50',
                                    type='nn_classifier',
                                    epochs=5)

In [ ]:
clf2.run()

In [ ]:
clf2.classifier.roc()

In [ ]:
clf2.classifier.confusion_matrix()

In [ ]:
clf2.classifier.show_model_layers()

In [ ]:
target_image_path = clf2.data_processor.test_table.iloc[4]['PATH']
target_image_path

In [ ]:
clf2.cam(target_image_path=target_image_path, target_layer=clf2.trained_model.layer4[2].conv3, alpha=0.3, type='scorecam', figure_size=(20,60))

In [ ]:
clf2.cam(target_image_path=target_image_path, target_layer=clf2.trained_model.layer4[2].conv3, alpha=0.3, type='gradcampp', figure_size=(20,60))

In [ ]:
clf2.data_processor.class_table()